<a href="https://colab.research.google.com/github/VbsmRobotic/dell-edge-ml-case/blob/main/Edge_Telemetry_Anomaly_Detection_Dell_CaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Edge_Telemetry_Anomaly_Detection

# Load a specific CSV file from Google Drive

In [1]:
# Install gdown to enable file downloading from Google Drive
!pip install -q gdown

# Import required libraries
import os                   # Provides utilities for interacting with the operating system
import pandas as pd         # Used for reading and analyzing CSV data
import gdown                # Simplifies downloading files from Google Drive

# Define constants
# FILE_ID = "https://drive.google.com/file/d/1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd/view?usp=sharing"  # Unique ID of the CSV file hosted on Google Drive
FILE_ID = "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd"  # Unique ID of the CSV file hosted on Google Drive
CSV_FILE = "2025-01-03.csv"                    # Local filename to save the downloaded CSV

# Download the file from Google Drive using gdown
print(f"Downloading CSV file with ID: {FILE_ID}")

# gdown will handle the URL generation and download the file to the given path
gdown.download(id=FILE_ID, output=CSV_FILE, quiet=False)

# Step 1: Read and print the header line from the CSV file
try:
    with open(CSV_FILE, 'r') as f:
        header_line = f.readline().strip()  # Read the first line and remove trailing newline characters
        print("\nCSV Header:\n", header_line)  # Display the CSV header
except Exception as e:
    print(f"Error reading header: {e}")  # Catch any error in reading the file and print it

# Step 2: Initialize an empty dictionary to store summary statistics
summary = {}

# Step 3: Try loading the full CSV file and analyzing its contents
try:
    # Load the CSV file into a DataFrame using pandas
    df = pd.read_csv(CSV_FILE)
    print(f"\nCSV loaded successfully with {len(df)} rows.")  # Print total number of rows in the CSV

    # Check if necessary columns exist in the file
    required_columns = {'failure', 'model'}
    if not required_columns.issubset(df.columns):
        # If either 'failure' or 'model' columns are missing, raise an error
        raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")

    # 🔧 Ensure failure column is clean and correctly typed
    df['failure'] = pd.to_numeric(df['failure'], errors='coerce').fillna(0).astype(int)

    # Filter the rows where 'failure' is equal to 1 (indicating a failure event)
    failure_rows = df[df['failure'] == 1].copy()  # Copy to avoid SettingWithCopyWarning

    # Add a column indicating the actual CSV file line number (including header)
    failure_rows['csv_line_number'] = failure_rows.index + 2  # +1 for 0-based index, +1 for header

    # Optional: Show a preview of the first few failure rows for inspection
    if not failure_rows.empty:
        print("\nPreview of failure rows with CSV line numbers:")
        print(failure_rows[['csv_line_number', 'date', 'serial_number', 'model']].head())  # Display preview

except Exception as e:
    # If any error occurs during loading or processing, capture it in the summary
    summary = {"error": str(e)}


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=58981190-1c8c-4a56-9b75-4d31c9553f6c
To: /content/2025-01-03.csv
100%|██████████| 119M/119M [00:00<00:00, 154MB/s]



CSV Header:
 date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,is_legacy_format,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_16_normalized,smart_16_raw,smart_17_normalized,smart_17_raw,smart_18_normalized,smart_18_raw,smart_22_normalized,smart_22_raw,smart_23_normalized,smart_23_raw,smart_24_normalized,smart_24_raw,smart_27_normalized,smart_27_raw,smart_71_normalized,smart_71_raw,smart_82_normalized,smart_82_raw,smart_90_normalized,smart_90_raw,smart_160_normalized,smart_160_raw,smart_161_normalized,smart_161_raw,smart_163_normalized,smart_163_raw,smart_164_normalized,s

# Random Forest classifier

In [ ]:
# 🧩 Install dependencies
!pip install -q gdown scikit-learn

# 📚 Import required libraries
import os                             # For file operations
import pandas as pd                   # For data handling
import gdown                          # To download CSVs from Google Drive
from sklearn.ensemble import RandomForestClassifier  # Random Forest model
from sklearn.metrics import classification_report    # Evaluation metrics
from sklearn.preprocessing import StandardScaler     # Normalize features

# 📂 Google Drive file IDs for each CSV (update these to match your real Drive links)
file_ids = {
    "2025-01-01.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-02.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-03.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-04.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-05.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-06.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-07.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd",
    "2025-01-08.csv": "1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd"
}

# 📥 Download files using gdown
def download_csv(filename, file_id):
    print(f"Downloading {filename} ...")
    gdown.download(id=file_id, output=filename, quiet=False)

for filename, file_id in file_ids.items():
    download_csv(filename, file_id)

# 🧪 Define dataset split
train_files = [f"2025-01-0{i}.csv" for i in range(1, 7)]  # Train: Jan 1–6
val_files = ["2025-01-07.csv"]                            # Validation: Jan 7
test_files = ["2025-01-08.csv"]                           # Test: Jan 8

# 📊 Read and combine CSV files
def load_data(file_list):
    dfs = []
    for file in file_list:
        df = pd.read_csv(file)
        df['failure'] = pd.to_numeric(df['failure'], errors='coerce').fillna(0).astype(int)
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

train_df = load_data(train_files)
val_df = load_data(val_files)
test_df = load_data(test_files)

# 🎯 Prepare features and labels
excluded_cols = ['date', 'serial_number', 'model', 'datacenter', 'cluster_id', 'vault_id', 'pod_id', 'pod_slot_num']
features = [col for col in train_df.columns if col not in excluded_cols + ['failure']]
X_train = train_df[features].fillna(0)
y_train = train_df['failure']
X_val = val_df[features].fillna(0)
y_val = val_df['failure']
X_test = test_df[features].fillna(0)
y_test = test_df['failure']

# 📏 Standardize feature values
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# 🧠 Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# 📈 Evaluate on validation data
val_preds = clf.predict(X_val)
print("📊 Validation Results:\n", classification_report(y_val, val_preds))

# 📈 Evaluate on test data
test_preds = clf.predict(X_test)
print("🧪 Test Results:\n", classification_report(y_test, test_preds))


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=e4aa2b4f-6527-4ade-ae27-62e55be357d2
To: /content/2025-01-01.csv
100%|██████████| 119M/119M [00:00<00:00, 214MB/s]


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=dc46b0bb-7612-4e75-ac7a-15d0dce35213
To: /content/2025-01-02.csv
100%|██████████| 119M/119M [00:00<00:00, 210MB/s]


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=adfddddb-099d-487d-a06d-c07524f42150
To: /content/2025-01-03.csv
100%|██████████| 119M/119M [00:00<00:00, 153MB/s]


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=1c581677-55c3-4ecc-9f05-e05870d97170
To: /content/2025-01-04.csv
100%|██████████| 119M/119M [00:00<00:00, 246MB/s] 


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=8cfbe864-57c8-4d21-b3db-85a67c564411
To: /content/2025-01-05.csv
100%|██████████| 119M/119M [00:00<00:00, 219MB/s]


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=7be8783e-9cf7-488f-9000-3b7efd98a659
To: /content/2025-01-06.csv
100%|██████████| 119M/119M [00:00<00:00, 192MB/s]


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=c644524f-2ea8-4341-bf91-65d799600e12
To: /content/2025-01-07.csv
100%|██████████| 119M/119M [00:00<00:00, 241MB/s] 


Downloading...
From (original): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd
From (redirected): https://drive.google.com/uc?id=1f1SA_MzgA9kwSBvs-stfyGzJE1yPaLsd&confirm=t&uuid=6e3bb5bb-a94b-49c9-9de3-541a2b8e24b5
To: /content/2025-01-08.csv
100%|██████████| 119M/119M [00:00<00:00, 185MB/s]
